In [1]:
%reload_ext autoreload
%autoreload 2

import time

start = time.time()
from torch_snippets import *
end = time.time()
print(end-start)

.loader 3.6728403568267822

.charts 0.18240594863891602

.paths 0.0018169879913330078

.markup 0.2538912296295166

.inspector 0.0007202625274658203

...

.pdf_lader 0.002458333969116211

.torch_loader 0.17966985702514648

.sklegos 0.5495610237121582

.imgaug_loader 0.1641850471496582

0.00012922286987304688